In [1]:
import torch as t
import warnings

warnings.filterwarnings("ignore")

t.Tensor().dtype

torch.float32

In [2]:
# 指定全局为某种类型的 Tensor

# t.set_default_tensor_type("torch.DoubleTensor")
# t.Tensor().dtype

In [3]:
import torchvision
import warnings

warnings.filterwarnings("ignore")

# 加载训练数据，参数 train=True，供 60000 条
train = torchvision.datasets.MNIST(
    root=".", train=True, transform=torchvision.transforms.ToTensor(), download=True
)
# 加载测试数据，参数 train=False，供 10000 条
test = torchvision.datasets.MNIST(
    root=".", train=False, transform=torchvision.transforms.ToTensor(), download=True
)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████| 9912422/9912422 [00:03<00:00, 2545715.94it/s]


Extracting .\MNIST\raw\train-images-idx3-ubyte.gz to .\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████| 28881/28881 [00:00<00:00, 558692.44it/s]


Extracting .\MNIST\raw\train-labels-idx1-ubyte.gz to .\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████| 1648877/1648877 [00:08<00:00, 190727.61it/s]


Extracting .\MNIST\raw\t10k-images-idx3-ubyte.gz to .\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|███████████████████████████████████| 4542/4542 [00:00<00:00, 4596026.24it/s]

Extracting .\MNIST\raw\t10k-labels-idx1-ubyte.gz to .\MNIST\raw



In [4]:
train.data.shape, train.targets.shape, test.data.shape, test.targets.shape

(torch.Size([60000, 28, 28]),
 torch.Size([60000]),
 torch.Size([10000, 28, 28]),
 torch.Size([10000]))

In [5]:
import torch

# 训练数据打乱，使用 64 小批量
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=64, shuffle=True)
# 测试数据无需打乱，使用 64 小批量
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=64, shuffle=False)
train_loader, test_loader

(<torch.utils.data.dataloader.DataLoader at 0x20cbc1a1890>,
 <torch.utils.data.dataloader.DataLoader at 0x20cb9dcbbd0>)

In [6]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 512)  # 784 是因为训练是我们会把 28*28 展平
        self.fc2 = nn.Linear(512, 128)  # 使用 nn 类初始化线性层（全连接层）
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))  # 直接使用 relu 函数，也可以自己初始化一个 nn 下面的 Relu 类使用
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # 输出层一般不激活
        return x
model = Net()
model

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [7]:
model(torch.randn(1, 784))

tensor([[-0.0027, -0.1050,  0.1214, -0.1501,  0.0639, -0.1103, -0.0867, -0.1116,
          0.0765,  0.0394]], grad_fn=<AddmmBackward0>)

In [10]:
loss_fn = nn.CrossEntropyLoss()  # 交叉熵损失函数
opt = torch.optim.Adam(model.parameters(), lr=0.002)  # Adam 优化器

In [11]:
def fit(epochs, model, opt):
    print("Start training, please be patient.")
    # 全数据集迭代 epochs 次
    for epoch in range(epochs):
        # 从数据加载器中读取 Batch 数据开始训练
        for i, (images, labels) in enumerate(train_loader):
            images = images.reshape(-1, 28 * 28)  # 对特征数据展平，变成 784
            labels = labels  # 真实标签
            outputs = model(images)  # 前向传播
            loss = loss_fn(outputs, labels)  # 传入模型输出和真实标签
            opt.zero_grad()  # 优化器梯度清零，否则会累计
            loss.backward()  # 从最后 loss 开始反向传播
            opt.step()  # 优化器迭代
            # 自定义训练输出样式
            if (i + 1) % 100 == 0:
                print(
                    "Epoch [{}/{}], Batch [{}/{}], Train loss: {:.3f}".format(
                        epoch + 1, epochs, i + 1, len(train_loader), loss.item()
                    )
                )
        # 每个 Epoch 执行一次测试
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.reshape(-1, 28 * 28)
            labels = labels
            outputs = model(images)
            # 得到输出最大值 _ 及其索引 predicted
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()  # 如果预测结果和真实值相等则计数 +1
            total += labels.size(0)  # 总测试样本数据计数
        print(
            "============ Test accuracy: {:.3f} =============".format(correct / total)
        )

In [12]:
fit(epochs=1, model=model, opt=opt)  # 训练 1 个 Epoch，预计持续 10 分钟

Start training, please be patient.
Epoch [1/1], Batch [100/938], Train loss: 0.324
Epoch [1/1], Batch [200/938], Train loss: 0.234
Epoch [1/1], Batch [300/938], Train loss: 0.088
Epoch [1/1], Batch [400/938], Train loss: 0.116
Epoch [1/1], Batch [500/938], Train loss: 0.153
Epoch [1/1], Batch [600/938], Train loss: 0.056
Epoch [1/1], Batch [700/938], Train loss: 0.123
Epoch [1/1], Batch [800/938], Train loss: 0.137
Epoch [1/1], Batch [900/938], Train loss: 0.121
============ Test accuracy: 0.969 =============


In [ ]:
# Sequential 容器结构

In [13]:
model_s = nn.Sequential(
    nn.Linear(784, 512),  # 线性类
    nn.ReLU(),  # 激活函数类
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Linear(128, 10),
)

model_s  # 查看网络结构

Sequential(
  (0): Linear(in_features=784, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=10, bias=True)
)

In [14]:
opt_s = torch.optim.Adam(model_s.parameters(), lr=0.002)  # Adam 优化器
fit(epochs=1, model=model_s, opt=opt_s)  # 训练 1 个 Epoch

Start training, please be patient.
Epoch [1/1], Batch [100/938], Train loss: 0.288
Epoch [1/1], Batch [200/938], Train loss: 0.410
Epoch [1/1], Batch [300/938], Train loss: 0.176
Epoch [1/1], Batch [400/938], Train loss: 0.136
Epoch [1/1], Batch [500/938], Train loss: 0.119
Epoch [1/1], Batch [600/938], Train loss: 0.210
Epoch [1/1], Batch [700/938], Train loss: 0.140
Epoch [1/1], Batch [800/938], Train loss: 0.063
Epoch [1/1], Batch [900/938], Train loss: 0.106
============ Test accuracy: 0.970 =============


In [15]:
torch.cuda.is_available()

False

In [16]:
# 如果 GPU 可用则使用 CUDA 加速，否则使用 CPU 设备计算
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
dev

device(type='cpu')

In [ ]:
# 数据和模型后面添加 .to(dev)，这样 PyTorch 就可以自动判断是否使用 GPU 加速
def fit(epochs, model, opt):
    print("Start training, please be patient.")
    for epoch in range(epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.reshape(-1, 28 * 28).to(dev)  # 添加 .to(dev)
            labels = labels.to(dev)  # 添加 .to(dev)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            opt.zero_grad()
            loss.backward()
            opt.step()
            if (i + 1) % 100 == 0:
                print(
                    "Epoch [{}/{}], Batch [{}/{}], Train loss: {:.3f}".format(
                        epoch + 1, epochs, i + 1, len(train_loader), loss.item()
                    )
                )
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.reshape(-1, 28 * 28).to(dev)  # 添加 .to(dev)
            labels = labels.to(dev)  # 添加 .to(dev)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        print(
            "============ Test accuracy: {:.3f} =============".format(correct / total)
        )

In [ ]:
# 模型保存
torch.save(model_s, "./model_s.pt")

In [ ]:
# 模型加载
model_s = torch.load("./model_s.pt")
model_s

In [ ]:
# 对测试数据第一个样本进行推理，注意将张量类型转换为 FloatTensor
result = model_s(test.data[0].reshape(-1, 28 * 28).type(torch.FloatTensor).to(dev))
torch.argmax(result)  # 找到输出最大值索引即为预测标签